In [1]:
from data_loading import load_base_data, load_siderophore, load_weight

In [2]:
meta, pfam, cath = load_base_data()

In [3]:
siderophore = load_siderophore(meta, pfam, cath)

In [4]:
weight = load_weight(meta, pfam, cath)

/home/nickbhat/projects/bgc_tran/data_loading.py:64: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return fam[fam.BGC.isin(metadata[idx].BGC)]


In [5]:
import numpy as np

In [6]:
np.max(meta.MW)

6771.0

In [10]:
meta.iloc[0]

BGC                  BGC0001311
Genus              Ruminococcus
Name                  flavecins
Compound_count                1
MW                           -1
TPSA                        NaN
LOGP                        NaN
HACC                        NaN
HDON                        NaN
Type                       RiPP
Activities        Antibacterial
Structure                   NaN
Gram                          1
Antibacterial                 1
Antifungal                    0
Siderophore                   0
Orf_length                   20
Name: 0, dtype: object

In [8]:
weight['cath']['gram+'][1]

0       NaN
1       1.0
4       0.0
6       1.0
8       0.0
       ... 
1857    1.0
1860    0.0
1861    NaN
1862    1.0
1863    1.0
Name: MW, Length: 1034, dtype: float64